In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 51.1 MB/s 
     |████████████████████████████████| 182 kB 72.5 MB/s 


In [3]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import re
import os
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [4]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [5]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [6]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#데이터 불러오기, 전처리

In [7]:
# data = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/concat_all_notTokenized.csv')
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_data.csv")
data = pd.read_csv('finance_data.csv')
data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
del data['sentence']

# 중복 제거
data.drop_duplicates(subset=['kor_sentence'], inplace=True)

In [8]:
data

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...
4841,2,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,0,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,2,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,2,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [9]:
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in data.kor_sentence]
sentences[:3]

['[CLS] Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다. [SEP]',
 '[CLS] 테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발할 계획이라고 성명은 밝혔다. [SEP]',
 '[CLS] 국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 해고와는 달리 회사는 사무직 직원 수를 줄였다고 일간 포스티메스가 보도했다. [SEP]']

In [10]:
labels = data['labels'].values
labels

array([0, 0, 2, ..., 2, 2, 2])

In [11]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요!')
print(result)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요', '!']


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [13]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다. [SEP]
['[CLS]', 'Gran', '##에', '따르면', ',', '그', '회', '##사는', '회', '##사가', '성', '##장', '##하고', '있는', '곳', '##이지', '##만', ',', '모든', '생', '##산', '##을', '러시아', '##로', '옮', '##길', '계', '##획', '##이', '없다', '##고', '한다', '.', '[SEP]']


In [14]:
MAX_LEN = 60 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [15]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [16]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [17]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [18]:
# X_data = data['kor_sentence']
# y_data = data['label']

In [19]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)

In [20]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)

In [21]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [22]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [23]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 데이터
labels = data['labels'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 5.모델 생성

In [24]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [25]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [26]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 1

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


#6.모델 학습

In [27]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [28]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...

  Average training loss: 0.82
  Training epcoh took: 0:00:51

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:02

Training complete!


# 7.테스트셋 평가

In [29]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    151.    Elapsed: 0:00:24.

Accuracy: 0.70
Test took: 0:00:38


# 8. 뉴스 기사 감성분석

필요한 함수 만들기

In [30]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 60

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [31]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

# 분석할 뉴스 가져오기

In [32]:
news_df = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/30days_kakao_news.csv')
news_df = news_df[['date','title']]
news_df.head()

,date,title
0,2022.08.01.,"강화군, SNS 통해 밀착 소통 앞장"
1,2022.08.01.,"은평구, ‘하반기 민방위 보충교육’ 실시"
2,2022.08.01.,"[마감] 삼성전자, 약보합세 마감...초등학교 입학 연령 하향 추진에 교육주 강..."
3,2022.08.01.,댄스듀오 라이언·춘식 돌아온다… 8월 컴백 예고
4,2022.08.01.,[현장EN:]질문하고 고찰하는 창작자 지코의 '개운한' 컴백


In [33]:
def predict(sentences):
  logits = test_sentences([sentences])
  if np.argmax(logits) == 0 :
    sent = "0"
  elif np.argmax(logits) == 1 :
    sent = "1"
  elif np.argmax(logits) == 2 :
    sent = "2"
    
  return sent

news_df['predict'] = 0

for i in range(len(news_df)):
  sent = predict(news_df.iloc[i,1])
  news_df['predict'][i] = sent

news_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,date,title,predict
0,2022.08.01.,"강화군, SNS 통해 밀착 소통 앞장",0
1,2022.08.01.,"은평구, ‘하반기 민방위 보충교육’ 실시",0
2,2022.08.01.,"[마감] 삼성전자, 약보합세 마감...초등학교 입학 연령 하향 추진에 교육주 강...",0
3,2022.08.01.,댄스듀오 라이언·춘식 돌아온다… 8월 컴백 예고,0
4,2022.08.01.,[현장EN:]질문하고 고찰하는 창작자 지코의 '개운한' 컴백,0
...,...,...,...
595,2022.08.30.,"대구학부모역량개발센터, 조부모ㆍ손자녀 행복레시피 개설",0
596,2022.08.30.,'중고거래 사기' 30대 피의자가 경찰과 6시간 대치 끝에 현관문을 사이에 두고...,0
597,2022.08.30.,금리인하요구권 실적 비교 공시…수용률 1위 농협은행,0
598,2022.08.30.,"홍천군, 심뇌혈관질환 예방 레드서클 캠페인 전개",0


전처리

In [34]:
# news_df = news_df[news_df['date'] != '3분 전']
# news_df = news_df[news_df['date'] != '5분 전']
# news_df = news_df[news_df['date'] != '2분 전']
# news_df = news_df[news_df['date'] != '4분 전']

# Results

Ratio of Sentiments (ALL DATA)

In [35]:
neutral = news_df[news_df['predict'] == 0]
positive = news_df[news_df['predict'] == 1]
negative = news_df[news_df['predict'] == 2]

# Neutral Ratio
Nt_R = len(neutral)/len(news_df)*100
# positive Ratio
Po_R = len(positive)/len(news_df)*100
# negative Ratio
Ne_R = len(negative)/len(news_df)*100

print('ratio of neutral:', Nt_R)
print('ratio of positive:', Po_R)
print('ratio of negative:', Ne_R)

ratio of neutral: 73.0
ratio of positive: 26.833333333333332
ratio of negative: 0.16666666666666669


### Ratio of Sentiment (PER DATE)

In [36]:
print('[Crawling Period]')
print(news_df['date'].iloc[0], '~',news_df['date'].iloc[-1])

[Crawling Period]
2022.08.01. ~ 2022.08.30.


Date to List

In [37]:
l = news_df['date'].value_counts()
l.to_frame()
date_list = list(l.index)
date_list

['2022.08.01.',
 '2022.08.02.',
 '2022.08.29.',
 '2022.08.28.',
 '2022.08.27.',
 '2022.08.26.',
 '2022.08.25.',
 '2022.08.24.',
 '2022.08.23.',
 '2022.08.22.',
 '2022.08.21.',
 '2022.08.20.',
 '2022.08.19.',
 '2022.08.18.',
 '2022.08.17.',
 '2022.08.16.',
 '2022.08.15.',
 '2022.08.14.',
 '2022.08.13.',
 '2022.08.12.',
 '2022.08.11.',
 '2022.08.10.',
 '2022.08.09.',
 '2022.08.08.',
 '2022.08.07.',
 '2022.08.06.',
 '2022.08.05.',
 '2022.08.04.',
 '2022.08.03.',
 '2022.08.30.']

Ratio of Sentiment (Date)

In [38]:
RatioSent_df = pd.DataFrame(columns=['date','neutral(%)','positive(%)','negative(%)'])
num = 0
for i in date_list:
  date_df = news_df[news_df['date'] == i]

  neutral = date_df[date_df['predict'] == 0]
  positive = date_df[date_df['predict'] == 1]
  negative = date_df[date_df['predict'] == 2]

  total = len(positive) + len(neutral) + len(negative)

  # Neutral Ratio
  Nt_R = len(neutral)/total*100
  # positive Ratio
  Po_R = len(positive)/total*100
  # negative Ratio
  Ne_R = len(negative)/total*100

  RatioSent_df.loc[num] = [i,Nt_R,Po_R,Ne_R]
  num = num + 1

RatioSent_df

,date,neutral(%),positive(%),negative(%)
0,2022.08.01.,80.0,20.0,0.0
1,2022.08.02.,60.0,40.0,0.0
2,2022.08.29.,85.0,15.0,0.0
3,2022.08.28.,75.0,25.0,0.0
4,2022.08.27.,80.0,20.0,0.0
5,2022.08.26.,70.0,30.0,0.0
6,2022.08.25.,75.0,25.0,0.0
7,2022.08.24.,85.0,15.0,0.0
8,2022.08.23.,90.0,10.0,0.0
9,2022.08.22.,60.0,40.0,0.0


# Weighted Average Sent From date

긍정 or 부정만 판단할 것이므로 중립의 비율은 무시.

부정보다 긍정의 비율이 다 높게 나오는 것으로 보임.

긍정 or 부정의 판단 방법:

if ) (부정비율) > 0 ⇒ 부정

if ) (부정비율) < 0 ⇒ 긍정

In [39]:
RatioSent_df['ratio'] = None
for i in range(len(RatioSent_df)):
  if RatioSent_df['positive(%)'].iloc[i] > 10 * float(RatioSent_df['negative(%)'].iloc[i]):
    RatioSent_df['ratio'].iloc[i] = RatioSent_df['positive(%)'].iloc[i]
  else:
    RatioSent_df['ratio'].iloc[i] = - float(RatioSent_df['negative(%)'].iloc[i])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [94]:
RatioSent_df['predict'] = None

for i in range(len(RatioSent_df)):
  if RatioSent_df['ratio'].iloc[i] > 35:
    RatioSent_df['predict'].iloc[i] = '긍정'
  else:
    RatioSent_df['predict'].iloc[i] = '부정'

RatioSent_df.head()

,predict,date,ratio
0,부정,2022/08/01,20.0
1,긍정,2022/08/02,40.0
2,부정,2022/08/29,15.0
3,부정,2022/08/28,25.0
4,부정,2022/08/27,20.0


## 감성데이터전처리

In [95]:
RatioSent_df['date'] = [s.replace('.','/') for s in RatioSent_df['date']]
RatioSent_df['date'] = [s.rstrip('/') for s in RatioSent_df['date']]
RatioSent_df = RatioSent_df[['predict','date','ratio']]

## 주가 데이터 불러오기

In [96]:
stock_df = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/카카오_30일주가데이터.csv',encoding='cp949')

In [97]:
# 필요한 데이터만
stock_df = stock_df[['일자','종가','대비','등락률','거래량']]
stock_df.rename(columns = {'일자' : 'date'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [107]:
stock_df

,date,종가,대비,등락률,거래량
0,2022/08/30,73200,1000,1.39,802825
1,2022/08/29,72200,-3800,-5.00,1632297
2,2022/08/26,76000,-100,-0.13,993727
3,2022/08/25,76100,1200,1.60,1252757
4,2022/08/24,74900,100,0.13,936191
5,2022/08/23,74800,-200,-0.27,1327312
6,2022/08/22,75000,-1700,-2.22,1250643
7,2022/08/19,76700,-2500,-3.16,1804047
8,2022/08/18,79200,-900,-1.12,1640906
9,2022/08/17,80100,-600,-0.74,1041971


# Date 기준으로 합치기 (뉴스 + 주가)

## 시도 1.

In [98]:
merge_df = pd.merge(RatioSent_df, stock_df, on='date')
merge_df

,predict,date,ratio,종가,대비,등락률,거래량
0,부정,2022/08/01,20.0,74000,-800,-1.07,907903
1,긍정,2022/08/02,40.0,71800,-2200,-2.97,1786920
2,부정,2022/08/29,15.0,72200,-3800,-5.00,1632297
3,부정,2022/08/26,30.0,76000,-100,-0.13,993727
4,부정,2022/08/25,25.0,76100,1200,1.60,1252757
5,부정,2022/08/24,15.0,74900,100,0.13,936191
6,부정,2022/08/23,10.0,74800,-200,-0.27,1327312
7,긍정,2022/08/22,40.0,75000,-1700,-2.22,1250643
8,긍정,2022/08/19,45.0,76700,-2500,-3.16,1804047
9,부정,2022/08/18,20.0,79200,-900,-1.12,1640906


In [99]:
merge_df[merge_df['predict']=='부정']

,predict,date,ratio,종가,대비,등락률,거래량
0,부정,2022/08/01,20.0,74000,-800,-1.07,907903
2,부정,2022/08/29,15.0,72200,-3800,-5.00,1632297
3,부정,2022/08/26,30.0,76000,-100,-0.13,993727
4,부정,2022/08/25,25.0,76100,1200,1.60,1252757
5,부정,2022/08/24,15.0,74900,100,0.13,936191
6,부정,2022/08/23,10.0,74800,-200,-0.27,1327312
9,부정,2022/08/18,20.0,79200,-900,-1.12,1640906
11,부정,2022/08/16,5.0,80700,-1800,-2.18,1798225
12,부정,2022/08/12,25.0,82500,-1200,-1.43,1069578
13,부정,2022/08/11,25.0,83700,3400,4.23,1918391


In [100]:
merge_df[merge_df['predict']=='긍정']

,predict,date,ratio,종가,대비,등락률,거래량
1,긍정,2022/08/02,40.0,71800,-2200,-2.97,1786920
7,긍정,2022/08/22,40.0,75000,-1700,-2.22,1250643
8,긍정,2022/08/19,45.0,76700,-2500,-3.16,1804047
10,긍정,2022/08/17,50.0,80100,-600,-0.74,1041971
19,긍정,2022/08/03,40.0,76000,4200,5.85,2893725


# 모델평가

In [101]:
merge_df['등락'] = None
for i in range(len(merge_df)):
  if merge_df['등락률'].iloc[i] >= 0:
    merge_df['등락'].iloc[i] = 1 #상승
  else:
    merge_df['등락'].iloc[i] = 0 #하락

merge_df['pre'] = None
for i in range(len(merge_df)):
  if merge_df['predict'].iloc[i] == '긍정':
    merge_df['pre'].iloc[i] = 1 #상승
  else:
    merge_df['pre'].iloc[i] = 0 #하락

merge_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,predict,date,ratio,종가,대비,등락률,거래량,등락,pre
0,부정,2022/08/01,20.0,74000,-800,-1.07,907903,0,0
1,긍정,2022/08/02,40.0,71800,-2200,-2.97,1786920,0,1
2,부정,2022/08/29,15.0,72200,-3800,-5.00,1632297,0,0
3,부정,2022/08/26,30.0,76000,-100,-0.13,993727,0,0
4,부정,2022/08/25,25.0,76100,1200,1.60,1252757,1,0


## 모델평가

In [102]:
merge_df['yes/no'] = None
for i in range(len(merge_df)):
  if merge_df['등락'].iloc[i] == merge_df['pre'].iloc[i]:
    merge_df['yes/no'].iloc[i] = 'yes' # 정답
  else:
    merge_df['yes/no'].iloc[i] = 'no' #오답
merge_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,predict,date,ratio,종가,대비,등락률,거래량,등락,pre,yes/no
0,부정,2022/08/01,20.0,74000,-800,-1.07,907903,0,0,yes
1,긍정,2022/08/02,40.0,71800,-2200,-2.97,1786920,0,1,no
2,부정,2022/08/29,15.0,72200,-3800,-5.00,1632297,0,0,yes
3,부정,2022/08/26,30.0,76000,-100,-0.13,993727,0,0,yes
4,부정,2022/08/25,25.0,76100,1200,1.60,1252757,1,0,no
5,부정,2022/08/24,15.0,74900,100,0.13,936191,1,0,no
6,부정,2022/08/23,10.0,74800,-200,-0.27,1327312,0,0,yes
7,긍정,2022/08/22,40.0,75000,-1700,-2.22,1250643,0,1,no
8,긍정,2022/08/19,45.0,76700,-2500,-3.16,1804047,0,1,no
9,부정,2022/08/18,20.0,79200,-900,-1.12,1640906,0,0,yes


CONFUSION MATRICS

In [103]:
TP = merge_df[(merge_df['등락']==1)&(merge_df['pre']==1)]
FP = merge_df[(merge_df['등락']==0)&(merge_df['pre']==1)]
FN = merge_df[(merge_df['등락']==1)&(merge_df['pre']==0)]
TN = merge_df[(merge_df['등락']==0)&(merge_df['pre']==0)]

In [104]:
cm = {'예측값(Y)':  [len(TP), len(FP)],
      '예측값(N)':  [len(FN), len(TN)],}
cmdf = pd.DataFrame(cm, index=['실제값(p)', '실제값(n)'], columns=['예측값(Y)', '예측값(N)'])
cmdf

,예측값(Y),예측값(N)
실제값(p),1,7
실제값(n),4,9


In [105]:
print("precision", len(TP)/(len(TP)+len(FP)))
print("recall", len(TP)/(len(TP)+len(FN)))
pc=len(TP)/(len(TP)+len(FP))
rc=len(TP)/(len(TP)+len(FN))
print("f1 score", 2*pc*rc/(pc+rc))

correct = merge_df[merge_df['yes/no'] == 'yes']
acc = len(correct)/len(merge_df)*100

print('Accuracy:', acc)

precision 0.2
recall 0.125
f1 score 0.15384615384615385
Accuracy: 47.61904761904761


CHI-SQUARE

In [106]:
from scipy.stats import chi2_contingency

cmdf.index = ['실제값(p)', '실제값(n)']

chiresult = chi2_contingency(cmdf, correction = False)
print('Chi square: {}'.format(chiresult[0]))
print('P-value: {}'.format(chiresult[1]))

Chi square: 0.9111778846153846
P-value: 0.3398020843351985


유의수준 0.1하에서 귀무가설 기각하고 대립가설을 수용.
<br/><br/>
H0(귀무가설) : 두 변수는 독립이다.
<br/>
HA(대립가설) : 두 변수는 독립이 아니다.

=> 두 변수는 독립이 아님.